# Burgers Equation Identification
Equation:   $u_{t} + \lambda_1 uu_{x}-\lambda_2 u_{xx} = 0$  

In [35]:
#author : $um@nth
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import grad
import scipy.io
from torch.utils.data import Dataset, DataLoader

In [36]:
N_u = 2500
layers = [2, 25, 25, 25, 25, 25, 25, 25, 25, 1]
data = scipy.io.loadmat('burgers_shock.mat') 
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))                  
u_star = Exact.flatten()[:,None]                                                 
lb = X_star.min(0)                                                              
ub = X_star.max(0)
np.random.seed(107)
idx = np.random.choice(X_star.shape[0], N_u, replace=False)

X_train = X_star[idx,:]
u_train = torch.from_numpy(u_star[idx,:]).float()
X = torch.from_numpy(X_train[:,0:1]).requires_grad_(True).float()     
T = torch.from_numpy(X_train[:,1:2]).requires_grad_(True).float()

In [37]:
class Data(Dataset):

  def __init__(self, X, T, u_train):
    self.x = torch.cat([X, T], axis=1).float()
    self.y = u_train
    self.len = u_train.shape[0]
  
  def __getitem__(self, index):    
      return self.x[index], self.y[index]
  
  def __len__(self):
      return self.len

In [38]:
data_set = Data(X, T, u_train)
train_loader = DataLoader(dataset=data_set, batch_size=128)

## PINN

In [39]:
class PINN(nn.Module):

  def __init__(self, layers):
    super(PINN, self).__init__()
    self.layers = nn.ModuleList()
    for i, j in zip(layers, layers[1:]):
      linear = nn.Linear(i, j)
      nn.init.xavier_normal_(linear.weight.data, gain = 1.0)
      nn.init.zeros_(linear.bias.data)
      self.layers.append(linear)
  
  def forward(self, x):
    L = len(self.layers)
    for l, transform in enumerate(self.layers):
      if l < L-1:
        x = torch.tanh(transform(x))
      else:
        x = transform(x)
    return x   

In [40]:
pinn_model = PINN(layers)
l1 = nn.Parameter(torch.tensor([0.], requires_grad=True))
l2 = nn.Parameter(torch.tensor([0.], requires_grad=True))
pinn_model.register_parameter('lambda_1', l1)
pinn_model.register_parameter('lambda_2', l2)
optimizer = torch.optim.Adam(pinn_model.parameters(), lr = 0.0005)

In [41]:
def residual_loss(X, T, model, l1, l2):
  xf = torch.cat([X, T], axis=1)
  uf = model(xf)
  u_x = grad(uf.sum(), X, retain_graph = True, create_graph = True)[0]
  u_xx = grad(u_x.sum(), X, retain_graph = True, create_graph = True)[0]
  u_t = grad(uf.sum(), T, retain_graph = True, create_graph = True)[0]
  f = u_t + l1*uf*u_x - l2*u_xx 
  return torch.mean(torch.square(f))

In [42]:
def PINN_train(model, train_loader, optimizer, X, T, l1, l2):
  epochs = 5000
  mse = nn.MSELoss()
  for epoch in range(epochs):
    for x1, y1 in train_loader:
      model.train()
      optimizer.zero_grad()
      yhat = model(x1)
      loss1 = mse(yhat, y1)
      loss2 = residual_loss(X, T, model, l1, l2)
      loss = loss1 + loss2
      loss.backward()
      optimizer.step()
    print('Epoch:', epoch, 'Loss: %.5e, Lambda_1: %.5f, Lambda_2: %.5f' % (loss.item(), l1, l2))
  return

In [43]:
PINN_train(pinn_model, train_loader, optimizer, X, T, l1, l2)

Epoch: 0 Loss: 2.83198e-01, Lambda_1: 0.00103, Lambda_2: 0.00742
Epoch: 1 Loss: 2.63034e-01, Lambda_1: -0.01043, Lambda_2: 0.01954
Epoch: 2 Loss: 2.50568e-01, Lambda_1: -0.01998, Lambda_2: 0.02999
Epoch: 3 Loss: 2.34797e-01, Lambda_1: -0.02561, Lambda_2: 0.03733
Epoch: 4 Loss: 2.09298e-01, Lambda_1: -0.02679, Lambda_2: 0.03754
Epoch: 5 Loss: 1.67750e-01, Lambda_1: -0.02235, Lambda_2: 0.02172
Epoch: 6 Loss: 1.18281e-01, Lambda_1: -0.01503, Lambda_2: 0.00111
Epoch: 7 Loss: 8.42037e-02, Lambda_1: -0.01617, Lambda_2: 0.00095
Epoch: 8 Loss: 6.49711e-02, Lambda_1: -0.02013, Lambda_2: 0.00027
Epoch: 9 Loss: 5.61211e-02, Lambda_1: -0.02596, Lambda_2: -0.00008
Epoch: 10 Loss: 5.32098e-02, Lambda_1: -0.03042, Lambda_2: -0.00046
Epoch: 11 Loss: 5.17561e-02, Lambda_1: -0.03118, Lambda_2: -0.00049
Epoch: 12 Loss: 5.08148e-02, Lambda_1: -0.02854, Lambda_2: -0.00041
Epoch: 13 Loss: 4.99802e-02, Lambda_1: -0.02412, Lambda_2: -0.00026
Epoch: 14 Loss: 4.92195e-02, Lambda_1: -0.01903, Lambda_2: -0.00007


In [58]:
lambda1 = float(l1.data)
lambda2 = float(l2.data)
print('Lambda1 Pred:', round(lambda1,8), '  ; Lambda1 Actual:', 1.0)
print('Lambda2 Pred:', round(lambda2,8), '; Lambda2 Actual:', round(0.01/np.pi,8))

Lambda1 Pred: 0.99360734   ; Lambda1 Actual: 1.0
Lambda2 Pred: 0.00395846 ; Lambda2 Actual: 0.0031831
